In [1]:
from qiskit import IBMQ
import qiskit
qiskit.__qiskit_version__

{'qiskit-terra': '0.10.0',
 'qiskit-aer': '0.3.2',
 'qiskit-ignis': '0.2.0',
 'qiskit-ibmq-provider': '0.3.3',
 'qiskit-aqua': '0.6.1',
 'qiskit': '0.13.0'}

In [2]:
provider = IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_vigo') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_ourense') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_london') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_burlington') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_essex') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_armonk') from IBMQ(hub='ibm-q', group='open', project='main')>]

In [3]:
from qiskit import *
from qiskit.compiler import transpile
from qiskit.visualization import plot_histogram
from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview
from qiskit.providers.aer import noise
import math

#Include our assertion function
from qiskit.compiler.assertion import classical_assertion, superposition_assertion, entanglement_assertion, calcSuccessrate

In [4]:
#we used ibmq_20_tokyo for our experiment, however, that backend has retired.  So we can use the ibmqx2 instead 
device = provider.get_backend('ibmqx2')
backend_monitor(device)
device

ibmqx2
Configuration
-------------
    n_qubits: 5
    operational: True
    status_msg: active
    pending_jobs: 0
    backend_version: 2.0.2
    basis_gates: ['u1', 'u2', 'u3', 'cx', 'id']
    local: False
    simulator: False
    allow_q_circuit: False
    conditional: False
    max_shots: 8192
    memory: True
    description: 5 qubit device
    backend_name: ibmqx2
    online_date: 2017-01-24T05:00:00+00:00
    n_registers: 1
    coupling_map: [[0, 1], [0, 2], [1, 0], [1, 2], [2, 0], [2, 1], [2, 3], [2, 4], [3, 2], [3, 4], [4, 2], [4, 3]]
    sample_name: sparrow
    open_pulse: False
    quantum_volume: 8
    max_experiments: 75
    allow_object_storage: True
    url: None
    allow_q_object: True
    credits_required: True

Qubits [Name / Freq / T1 / T2 / U1 err / U2 err / U3 err / Readout err]
-----------------------------------------------------------------------
    Q0 / 5.28627 GHz / 48.9011 µs / 47.74382 µs / 0.0 / 0.00068 / 0.00136 / 0.015
    Q1 / 5.23787 GHz / 75.88025 µ

<IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q', group='open', project='main')>

In [5]:
pi = math.pi
transQcList = []

In [6]:
def BVCircuit(bvCircuit, nbits, hiddenValue, qr, cr):
    for i in range(nbits):
        bvCircuit.h(qr[i])
    bvCircuit.x(qr[nbits])
    bvCircuit.h(qr[nbits])
    
    # Apply the inner-product oracle
    for j in range(nbits):
        if (hiddenValue & (1 << j)):
            bvCircuit.cx(qr[j], qr[nbits])
#        else:
#            bvCircuit.iden(qr[j])
            
    for i in range(nbits):
        bvCircuit.h(qr[i])
        
    for i in range(nbits):
        bvCircuit.measure(qr[i], cr[i])

In [7]:
def BV2():
    qr = QuantumRegister(3)
    # for recording the measurement on qr
    cr = ClassicalRegister(2)

    circuitName = "BV2"
    circ = QuantumCircuit(qr, cr)
    BVCircuit(circ, 2, 2, qr, cr)
    trans_qc = transpile(circ, device, optimization_level = 3)
    #print("BV2 size =",trans_qc.size(),"depth = ",trans_qc.depth(), "width = ",trans_qc.width(), "\n OPs =", trans_qc.count_ops(), "subcircuits =",trans_qc.num_tensor_factors())
    num_cnots = trans_qc.count_ops()['cx']
    return trans_qc, num_cnots

def BV2_assertion():
    #assertion
    qr = QuantumRegister(3)
    # for recording the measurement on qr
    cr = ClassicalRegister(2)

    circuitName = "BV2_assertion"
    circ = QuantumCircuit(qr, cr)
    BVCircuit(circ, 2, 2, qr, cr)
    qubitList = [qr[2]]
    phaseDict = {qr[2]:[pi/2, pi]}
    superposition_assertion(circ, qubitList, phaseDict, flag = 1)
    trans_qc = transpile(circ, device, optimization_level = 3)
    #print("BV2_assertion size =",trans_qc_assert.size(),"depth = ",trans_qc_assert.depth(), "width = ",trans_qc_assert.width(), "\n OPs =", trans_qc_assert.count_ops(), "subcircuits =",trans_qc_assert.num_tensor_factors())
    num_cnots = trans_qc.count_ops()['cx']
    return trans_qc, num_cnots

In [8]:
def BV2_experiment():
    #This function run the transpiler multiple times and selects the mapping with fewest SWAP gates
    min_cnots = -1
    for i in range(10):
        trans_qc, num_cnots = BV2()
        if min_cnots < 0:
            min_cnots = num_cnots
        elif (num_cnots < min_cnots) and (min_cnots >= 0):
            min_cnots = num_cnots
    while 1:
        trans_qc, num_cnots = BV2()
        if num_cnots == min_cnots:
            print("number of CNOTs in circuit without assertion: ", num_cnots)
            transQcList.append(trans_qc)
            break
            
    min_cnots = -1
    for i in range(20):
        trans_qc, num_cnots = BV2_assertion()
        if min_cnots < 0:
            min_cnots = num_cnots
        elif (num_cnots < min_cnots) and (min_cnots >= 0):
            min_cnots = num_cnots
    while 1:
        trans_qc, num_cnots = BV2_assertion()
        if num_cnots == min_cnots:
            print("number of CNOTs in circuit with assertion: ", num_cnots)
            transQcList.append(trans_qc)
            break

In [9]:
def BV3():
    #3bits
    qr = QuantumRegister(4)
    # for recording the measurement on qr
    cr = ClassicalRegister(3)

    circuitName = "BV3"
    circ= QuantumCircuit(qr, cr)
    BVCircuit(circ, 3, 6, qr, cr)
    trans_qc = transpile(circ, device)
    #print("BV3 size =",trans_qc.size(),"depth = ",trans_qc.depth(), "width = ",trans_qc.width(), "\n OPs =", trans_qc.count_ops(), "subcircuits =",trans_qc.num_tensor_factors())
    num_cnots = trans_qc.count_ops()['cx']
    return trans_qc, num_cnots

def BV3_assertion():
    #assertion
    qr = QuantumRegister(4)
    # for recording the measurement on qr
    cr = ClassicalRegister(3)
    circuitName = "BV3_assertion"
    circ = QuantumCircuit(qr, cr)
    BVCircuit(circ, 3, 6, qr, cr)
    qubitList = [qr[3]]
    phaseDict = {qr[3]:[pi/2, pi]}
    superposition_assertion(circ, qubitList, phaseDict, flag = 1)
    trans_qc = transpile(circ, device)
    #print("BV3_assertion size =",trans_qc_assert.size(),"depth = ",trans_qc_assert.depth(), "width = ",trans_qc_assert.width(), "\n OPs =", trans_qc_assert.count_ops(), "subcircuits =",trans_qc_assert.num_tensor_factors())
    num_cnots = trans_qc.count_ops()['cx']
    return trans_qc, num_cnots

In [10]:
def BV3_experiment():
    #This function run the transpiler multiple times and selects the mapping with fewest SWAP gates
    min_cnots = -1
    for i in range(10):
        trans_qc, num_cnots = BV3()
        if min_cnots < 0:
            min_cnots = num_cnots
        elif (num_cnots < min_cnots) and (min_cnots >= 0):
            min_cnots = num_cnots
    while 1:
        trans_qc, num_cnots = BV3()
        if num_cnots == min_cnots:
            print("number of CNOTs in circuit without assertion: ", num_cnots)
            transQcList.append(trans_qc)
            break
            
    min_cnots = -1
    for i in range(20):
        trans_qc, num_cnots = BV3_assertion()
        if min_cnots < 0:
            min_cnots = num_cnots
        elif (num_cnots < min_cnots) and (min_cnots >= 0):
            min_cnots = num_cnots
    while 1:
        trans_qc, num_cnots = BV3_assertion()
        if num_cnots == min_cnots:
            print("number of CNOTs in circuit with assertion: ", num_cnots)
            transQcList.append(trans_qc)
            break

In [11]:
#BV4 with assertion requires 6 qubits, therefore it is not executable on ibmqx2 
# def BV4():
#     #4bits
#     qr = QuantumRegister(5)
#     # for recording the measurement on qr
#     cr = ClassicalRegister(4)

#     circuitName = "BV4"
#     circ = QuantumCircuit(qr, cr)
#     BVCircuit(circ, 4, 14, qr, cr)
#     trans_qc = transpile(circ, device, optimization_level = 3)
#     transQcList.append(trans_qc)
#     #print("BV4 size =",trans_qc.size(),"depth = ",trans_qc.depth(), "width = ",trans_qc.width(), "\n OPs =", trans_qc.count_ops(), "subcircuits =",trans_qc.num_tensor_factors())
#     num_cnots = trans_qc.count_ops()['cx']
#     return trans_qc, num_cnots

# def BV4_assertion():
#     #assertion
#     qr = QuantumRegister(5)
#     # for recording the measurement on qr
#     cr = ClassicalRegister(4)

#     circuitName = "BV4_assertion"
#     circ = QuantumCircuit(qr, cr)
#     BVCircuit(circ, 4, 14, qr, cr)
#     qubitList = [qr[4]]
#     phaseDict = {qr[4]:[pi/2, pi]}
#     superposition_assertion(circ, qubitList, phaseDict, flag = 1)
#     trans_qc = transpile(circ, device, optimization_level = 3)
#     #print("BV4_assertion size =",trans_qc.size(),"depth = ",trans_qc.depth(), "width = ",trans_qc.width(), "\n OPs =", trans_qc.count_ops(), "subcircuits =",trans_qc.num_tensor_factors())
#     num_cnots = trans_qc.count_ops()['cx']
#     return trans_qc, num_cnots

In [12]:
#BV4 with assertion requires 6 qubits, therefore it is not executable on ibmqx2 
# def BV4_experiment():
#     #This function run the transpiler multiple times and selects the mapping with fewest SWAP gates
#     min_cnots = -1
#     for i in range(10):
#         trans_qc, num_cnots = BV4()
#         if min_cnots < 0:
#             min_cnots = num_cnots
#         elif (num_cnots < min_cnots) and (min_cnots >= 0):
#             min_cnots = num_cnots
#     while 1:
#         trans_qc, num_cnots = BV4()
#         if num_cnots == min_cnots:
#             print("number of CNOTs in circuit without assertion: ", num_cnots)
#             transQcList.append(trans_qc)
#             break
            
#     min_cnots = -1
#     for i in range(20):
#         trans_qc, num_cnots = BV4_assertion()
#         if min_cnots < 0:
#             min_cnots = num_cnots
#         elif (num_cnots < min_cnots) and (min_cnots >= 0):
#             min_cnots = num_cnots
#     while 1:
#         trans_qc, num_cnots = BV4_assertion()
#         if num_cnots == min_cnots:
#             print("number of CNOTs in circuit with assertion: ", num_cnots)
#             transQcList.append(trans_qc)
#             break

In [13]:
BV2_experiment()
BV3_experiment()
#BV4_experiment()

number of CNOTs in circuit without assertion:  1
number of CNOTs in circuit with assertion:  2
number of CNOTs in circuit without assertion:  5
number of CNOTs in circuit with assertion:  7


In [14]:
shots = 8192
#execute the job on real device
job = execute(transQcList, backend=device, shots=shots)
job_monitor(job)
res = job.result()

Job Status: job has successfully run


Success rate of BV2 without assertion

In [15]:
calcSuccessrate(res.get_counts(0), ["10"], 0)

total_count =  8192 success_count =  6943 success_rate =  84.75341796875 %


Success rate of BV2 with assertion

In [16]:
calcSuccessrate(res.get_counts(1), ["0 10"], 1)

total_count =  7866 success_count =  7124 success_rate =  90.56699720315281 %


Success rate of BV3 without assertion

In [17]:
calcSuccessrate(res.get_counts(2), ["110"], 0)

total_count =  8192 success_count =  6650 success_rate =  81.1767578125 %


Success rate of BV3 with assertion

In [18]:
calcSuccessrate(res.get_counts(3), ["0 110"], 1)

total_count =  7581 success_count =  5644 success_rate =  74.44928109748055 %
